In [1]:
require 'hdf5';
require 'nn';
require 'cutorch';
require 'cunn';
require 'torch';

In [2]:
myFiletrian=hdf5.open('train.h5','r')
myFiletest=hdf5.open('test.h5','r')

In [6]:
trainset={data=myFiletrian:read('data'):all(),label=myFiletrian:read('label'):all():byte()}

In [3]:
testset={data=myFiletest:read('data'):all(),label=myFiletest:read('label'):all():byte()}

In [9]:
setmetatable(trainset, 
    {__index = function(t, i) 
                    return {t.data[i], t.label[i]} 
                end}
);
function trainset:size()
    return self.data:size(1)
end
trainset.data=trainset.data:cuda()
trainset.label=trainset.label:cuda()
testset.label=testset.label:cuda()
testset.data=testset.data:cuda()

In [16]:
net=nn.Sequential()
net:add(nn.VolumetricConvolution(3,32,5,5,5,2,2,2,0,0,0))
net:add(nn.VolumetricConvolution(32,32,3,3,3,1,1,1,0,0,0))
net:add(nn.VolumetricMaxPooling(2,2,2,2,2,2))
net:add(nn.View(32*6*6*6))
net:add(nn.Linear(32*6*6*6,128))
net:add(nn.Linear(128,10))
net:add(nn.LogSoftMax())


In [19]:
criterion=nn.ClassNLLCriterion()
trainer=nn.StochasticGradient(net,criterion)
trainer.learningRate=0.0002
trainer.maxIteration=30
criterion=criterion:cuda()
net=net:cuda()

In [41]:
trainer:train(trainset)

# StochasticGradient: training	


# current error = 0.20806747223642	


# current error = 0.20218099710133	


# current error = 0.19785047711208	


# current error = 0.19435925871161	


...

# current error = 0.13698690986589	


# current error = 0.13475312222404	


# current error = 0.13255354995983	


# current error = 0.13036666847662	


# current error = 0.12820724563695	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.12820724563695	



In [42]:
correct = 0
for i=1,1956 do
    local groundtruth = testset.label[i][1]
    --print (groundtruth)
    local prediction = net:forward(testset.data[i])
    local confidences, indices = torch.sort(prediction, true)
    --print (confidences)
    --print ('--------------')
    --print (indeices)
    if groundtruth == indices[1] then
       correct = correct + 1 
    end
end
print (correct)
--print(correct, 100*correct/3500 .. ' % ')
print(correct, 100*correct/1956 .. ' % ')

1453	
1453	74.284253578732 % 	


In [21]:
testset

{
  data : CudaTensor - size: 1956x3x32x32x32
  label : CudaTensor - size: 1956x1
}


2	
